In [1]:
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv
import os
import mysql.connector
load_dotenv()

db_user = os.getenv("db_user")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")
db_port = os.getenv("db_port")
db_name = "jiohotstar_ads"

db_path = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

db = SQLDatabase.from_uri(db_path)
print(db.get_usable_table_names())

['brands', 'matches']


In [2]:
db.dialect

'mysql'

In [4]:
print(db.table_info)


CREATE TABLE brands (
	match_id BIGINT, 
	total_frames BIGINT, 
	frame_no BIGINT, 
	brand TEXT, 
	brand_position TEXT, 
	time_sec DOUBLE, 
	duration_sec DOUBLE, 
	confidence DOUBLE, 
	frame_path TEXT, 
	`Created_at` TEXT
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from brands table:
match_id	total_frames	frame_no	brand	brand_position	time_sec	duration_sec	confidence	frame_path	Created_at
1	258	1	KFC	Ground	0.0400000000	0.0400000000	0.9640000000	data\1\extracted_frames\KFC\frame_000001.jpg	2025-11-27 10:36:35
1	258	2	KFC	Ground	0.0800000000	0.0400000000	0.9360000000	data\1\extracted_frames\KFC\frame_000002.jpg	2025-11-27 10:36:35
1	258	3	KFC	Ground	0.1200000000	0.0400000000	0.9280000000	data\1\extracted_frames\KFC\frame_000003.jpg	2025-11-27 10:36:35
*/


CREATE TABLE matches (
	match_id INTEGER NOT NULL, 
	teams VARCHAR(255), 
	location VARCHAR(255), 
	match_type VARCHAR(100), 
	winner VARCHAR(255), 
	video_path VARCHAR(255), 
	match_timestamp DATETIME

In [4]:
import pandas as pd

In [8]:
from database import Query_a_Table
Q = "SELECT * FROM matches"
Query_a_Table(Q)


ModuleNotFoundError: No module named 'database'

In [6]:
import ollama

model_name = "llama3.2:1b"  # replace with your actual model name
messages = [{"role": "system", 
    "content": f"You are a sql query generator. Generate a sql query based on the user input. only return the sql query. Tabel info is given below: {db.get_table_info()}"}]

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    messages.append({"role": "user", "content": user_input})
    
    response = ollama.chat(model=model_name, messages=messages)
    bot_reply = response.message.content
    result = db.run(bot_reply)
    print("Bot:", result)
    
    messages.append({"role": "assistant", "content": bot_reply})


Bot: [(1, 'IN Vs AUS', 'IN', 'T50', 'NZ', 'data/1/source_video.mp4', datetime.datetime(2025, 11, 11, 2, 0), datetime.datetime(2025, 11, 24, 7, 55, 50)), (2, 'IN Vs AUS', 'IN', 'T50', 'NZ', 'data/2/source_video.mp4', datetime.datetime(2025, 11, 11, 2, 0), datetime.datetime(2025, 11, 24, 7, 58, 21))]
Bot: [(1, 1, 0.04, 258, 'KFC', 0.964, 114, 151, 333, 365), (1, 2, 0.08, 258, 'KFC', 0.936, 211, 150, 423, 363), (1, 3, 0.12, 258, 'KFC', 0.928, 300, 149, 509, 359), (1, 4, 0.16, 258, 'KFC', 0.854, 387, 146, 595, 354), (1, 5, 0.2, 258, 'KFC', 0.792, 468, 142, 678, 349), (1, 6, 0.24, 258, 'KFC', 0.872, 546, 139, 757, 343), (1, 7, 0.28, 258, 'KFC', 0.981, 621, 134, 830, 337), (1, 8, 0.32, 258, 'KFC', 0.977, 693, 127, 847, 323), (1, 9, 0.36, 258, 'KFC', 0.839, 758, 118, 847, 322), (1, 10, 0.4, 258, 'KFC', 0.205, 813, 111, 848, 318), (1, 39, 1.56, 258, 'Toyota', 0.842, 117, 63, 848, 170), (1, 40, 1.6, 258, 'Toyota', 0.821, 119, 65, 848, 169), (1, 41, 1.64, 258, 'Toyota', 0.824, 119, 65, 848, 169)

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1054 (42S22): Unknown column 'm.time_sec' in 'field list'
[SQL: SELECT m.match_id, b.brand, m.time_sec
FROM matches m
JOIN brand_detections b ON m.match_id = b.match_id]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [31]:
import google.generativeai as genai
# Configure your Gemini API key (set this securely or via environment variable)
genai.configure(api_key=os.getenv("GOOGLE_GENAI_API_KEY"))

# Initialize Gemini Pro model for chat
model = genai.GenerativeModel("gemini-2.5-flash")
messages = [{"role": "system", 
    "content": f"You are a sql query generator. Generate a sql query based on the user input. only return the sql query. Tabel info is given below: {db.get_table_info()}"}]

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    messages.append({"role": "user", "content": user_input})
    
    response = ollama.chat(model=model_name, messages=messages)
    bot_reply = response.message.content
    result = db.run(bot_reply)
    print("Bot:", result)
    
    messages.append({"role": "assistant", "content": bot_reply})


Bot: [(1, 'IN Vs Aus', 'IN', 'T20', 'IN', 'data/1/source_video.mp4', datetime.datetime(2025, 11, 8, 2, 0), datetime.datetime(2025, 11, 19, 8, 46, 8)), (2, 'NZ vs AUS', 'NZ', 'T50', 'NZ', 'data/2/source_video.mp4', datetime.datetime(2025, 11, 11, 2, 0), datetime.datetime(2025, 11, 23, 7, 53, 34))]
Bot: [('IN',), ('NZ',)]
Bot: [('T20',), ('T50',)]


Usable tables: ['matches']


In [19]:

# Interactive loop: user inputs NL question → generate SQL → execute → print results
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    messages.append({"role": "user", "content": user_input})

    # Generate SQL query with Ollama LLM
    response = ollama.chat(model=model_name, messages=messages)
    sql_query = response.message.content.strip()

    print("Generated SQL:", sql_query)

    # Run generated SQL against DB safely (you can add validation here)
    try:
        result = db.run(sql_query)
        print("Result:", generate_natural_language_answer(sql_query,result))
    except Exception as e:
        print("SQL execution error:", e)

    # Append the assistant message (SQL query generated) to keep context
    messages.append({"role": "assistant", "content": sql_query})


Generated SQL: SELECT * FROM matches;
SQL execution error: generate_natural_language_answer() missing 3 required positional arguments: 'sql', 'result', and 'llm'


In [20]:
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv
import os
import json
import ollama

load_dotenv()

# Load DB credentials from environment
db_user = os.getenv("db_user")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")
db_port = os.getenv("db_port")
db_name = "jiohotstar_ads"

# Build DB URI and initialize LangChain SQLDatabase
db_path = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
db = SQLDatabase.from_uri(db_path)

# Get table info as schema context
schema_info = db.get_table_info()

# Initialize Ollama client and model name
model_name = "llama3.2:1b"

# Initialize chat messages with system role and schema information
messages = [
    {
        "role": "system",
        "content": (
            "You are a SQL query generator. "
            "Generate a valid SQL SELECT query based on the user input. "
            "Only return the SQL query. "
            "Table info is given below:\n" + schema_info
        ),
    }
]

print("Usable tables:", db.get_usable_table_names())

answer_prompt_template = """
You are a helpful data analyst.

Original question:
{question}

SQL query executed:
{sql}

Query result (showing first few rows):
{result}

Using ONLY the information above, answer the question clearly and concisely.
If the result does not provide enough info, say you cannot answer confidently.
"""

def generate_natural_language_answer(question, sql, result, llm, max_rows=10):
    # Convert result rows to short JSON string for prompt
    trimmed_result = json.dumps(result[:max_rows], indent=2, default=str)

    prompt = answer_prompt_template.format(question=question, sql=sql, result=trimmed_result)
    messages = [
        {"role": "system", "content": "You are a helpful data analyst."},
        {"role": "user", "content": prompt}
    ]
    response = llm.chat(model=model_name, messages=messages)
    return response.message.content.strip()

Usable tables: ['matches']


In [21]:

# Interactive loop: user inputs NL question → generate SQL → execute → print NL results
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    messages.append({"role": "user", "content": user_input})

    # Generate SQL query with Ollama LLM
    response = ollama.chat(model=model_name, messages=messages)
    sql_query = response.message.content.strip()
    print("Generated SQL:", sql_query)

    # Run generated SQL against DB safely
    try:
        result = db.run(sql_query)
        # Generate natural language answer from SQL result
        nl_answer = generate_natural_language_answer(user_input, sql_query, result, ollama)
        print("Bot (natural language answer):", nl_answer)
    except Exception as e:
        print("SQL execution error:", e)

    # Append the assistant message (SQL query generated) to keep context
    messages.append({"role": "assistant", "content": sql_query})


Generated SQL: SELECT * FROM matches
Bot (natural language answer): I can help you with that.

You should see a table with one or more columns. Here is an example of what it might look like:
```
+----------+------------+
| MatchID | Player1   | ...       |
+----------+------------+
```

This shows all records in the `matches` table, including all columns (`*`).
Generated SQL: SELECT * FROM matches LIMIT 1
Bot (natural language answer): I can help with that. Since we are dealing with a database table called "matches", the query `SELECT * FROM matches LIMIT 1` will return only one row from this table.


Given the provided SQL query results, it seems like they are displaying the first few rows of the "matches" table. To confirm if there is indeed just one row to be displayed in the "matches" table, we would need more information about the data type or specific characteristics that might influence how many rows are returned.

However, assuming the result shows a single match record with re

In [ ]:
"""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run,
then look at the results of the query and return the answer. Unless the user
specifies a specific number of examples they wish to obtain, always limit your
query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting
examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while
executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.
""".format(
    dialect=db.dialect,
    top_k=5,
)